# Data Extraction 

In [3]:
import pandas as pd
from pymongo import MongoClient
import os
from dotenv import load_dotenv

In [4]:
# Cargar variables desde .env
load_dotenv()

def get_com_sinco() -> pd.DataFrame:
    """
    Connect to MongoDB databases (DB20 and DB25) and retrieve communications data.

    Returns
    -------
    pd.DataFrame
        Combined DataFrame with columns: code, category, messages.
    """
    uri = os.getenv("MONGO_URI")
    db20 = os.getenv("DB_NAME_20")
    db25 = os.getenv("DB_NAME_25")

    # Database connection
    con = MongoClient(uri)
    db_20 = con[db20]
    db_25 = con[db25]

    # Get com sinco 20
    q = {}
    p = {"_id": 0, "correlativo": 1, "asunto_id2/name": 1, "comunicacion": 1}
    cursor = db_20["comunication"].find(q, p)
    df_20 = pd.DataFrame.from_records(cursor)
    df_20.rename(columns={
        "correlativo": "code",
        "asunto_id2/name": "category",
        "comunicacion": "messages"
    }, inplace=True)

    # Get com sinco 25
    q = {}
    p = {"_id": 0, "code": 1, "categogory_id": 1, "messages": 1}
    cursor = db_25["com.caso"].find(q, p)
    df_25 = pd.DataFrame.from_records(cursor)
    df_25["category"] = df_25.apply(
        lambda x: x["categogory_id"][1] if x["categogory_id"] else "", axis=1
    )
    df_25.drop(columns="categogory_id", inplace=True)

    # Combine both datasets
    df = pd.concat([df_20, df_25])
    return df


In [ ]:
df = get_com_sinco()

In [8]:
df.to_excel('datos_sinco.xlsx', index=False)